<a href="https://colab.research.google.com/github/kohsuke2626/kohsuke2626.github.io/blob/main/(%E6%A3%9A%E5%8D%B8%E3%81%97%E7%94%A8)ShipStation%E5%87%BA%E8%8D%B7%E5%AE%9F%E7%B8%BE%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1) 必要ライブラリ ---
import pandas as pd

# --- 2) Colabでファイルアップロード ---
from google.colab import files
uploaded = files.upload()  # ShipStation.xlsx を選択

# アップロードしたファイル名を取得（1つ想定）
xlsx_path = next(iter(uploaded.keys()))
print("Loaded:", xlsx_path)

# --- 3) Excel読込（最初のシート） ---
xl = pd.ExcelFile(xlsx_path)
sheet = xl.sheet_names[0]
df = pd.read_excel(xlsx_path, sheet_name=sheet)

# --- 4) 店舗（セクション）見出し行を検出 ---
# 「Items Shipped」だけに値があり、他列が全部NaNの行を店舗名行とみなす
store_rows = df[
    df["Items Shipped"].notna() &
    df.drop(columns=["Items Shipped"]).isna().all(axis=1)
].copy()

store_rows["store"] = store_rows["Items Shipped"].astype(str).str.strip()

# 対象店舗（Wholesale除外）
targets = {
    "TENGA USA",
    "Official USA TENGA Online Store",
    "iroha Store USA",
}
exclude = {"TENGA USA Wholesale"}

# --- 5) セクション範囲を作る（店舗名行～次の店舗名行まで） ---
store_indices = store_rows.index.tolist()
stores = store_rows["store"].tolist()

sections = []
for i, (idx, store) in enumerate(zip(store_indices, stores)):
    start = idx + 1
    end = store_indices[i + 1] if i + 1 < len(store_indices) else len(df)
    sections.append((store, start, end))

# --- 6) セクションから SKU / Qty Sold を抽出する関数 ---
def extract_section(df, start, end):
    sub = df.iloc[start:end].copy()
    sub = sub[sub.notna().any(axis=1)]  # 空行除外

    # ヘッダ行（Sku / Alias Sku ...）を除外
    sub = sub[~sub["Items Shipped"].astype(str).str.strip().str.lower().eq("sku")]

    # アイテム行だけ拾う（Alias Skuが埋まってる行を商品行とみなす）
    sub = sub[sub["Unnamed: 1"].notna()]

    out = pd.DataFrame({
        "SKU": sub["Items Shipped"].astype(str).str.strip(),
        "Qty Sold": pd.to_numeric(sub["Unnamed: 4"], errors="coerce")
    }).dropna(subset=["Qty Sold"])

    out["Qty Sold"] = out["Qty Sold"].astype(int)

    # 念のため変なSKUを除外
    out = out[out["SKU"].str.lower() != "nan"]
    out = out[out["SKU"] != ""]
    return out

# --- 7) 対象店舗だけ結合して、SKU別に合算 ---
rows = []
for store, start, end in sections:
    store_clean = str(store).strip()
    if store_clean in exclude:
        continue
    if store_clean in targets:
        rows.append(extract_section(df, start, end))

if not rows:
    raise ValueError("対象店舗のデータが見つかりませんでした。店舗名が想定と違う可能性があります。")

combined = pd.concat(rows, ignore_index=True)

# SKUを整形（大文字化など）
combined["SKU"] = combined["SKU"].astype(str).str.strip().str.upper()

result = (
    combined.groupby("SKU", as_index=False)["Qty Sold"]
    .sum()
    .sort_values("Qty Sold", ascending=False)
    .reset_index(drop=True)
)

# --- 8) 出力（DataFrame表示） ---
print("Rows:", len(result))
display(result.head(20))

# --- 9) CSV / Excelで保存してダウンロード ---
out_csv = "SKU_QtySold_TENGA_Official_iroha.csv"
out_xlsx = "SKU_QtySold_TENGA_Official_iroha.xlsx"

result.to_csv(out_csv, index=False, encoding="utf-8-sig")
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    result.to_excel(writer, index=False, sheet_name="SKU_Summary")

files.download(out_csv)
files.download(out_xlsx)


Saving ShipStation.xlsx to ShipStation.xlsx
Loaded: ShipStation.xlsx
Rows: 132


,SKU,Qty Sold
0,TLO-002,168
1,TLO-002R,164
2,TFT-001,127
3,TFT-002,50
4,POT-005,33
5,OTENGA,22
6,STI-141,21
7,TOC-201PT,19
8,TOC-018RC,19
9,TLO-003,14


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>